In [1]:
import pandas as pd

from src.extraction.jsonl_data_reader import JsonlDataReader

train_data = JsonlDataReader(file_name='train.jsonl').read()
dev_data = JsonlDataReader(file_name='dev.jsonl').read()
test_data = JsonlDataReader(file_name='test.jsonl').read()

stage_order = ['pretokenizer', 'tokenizer', 'post_tokenizer', 'vectorizer', 'model']
stage_order_map = dict(zip(stage_order[:-1], stage_order[1:]))



In [2]:
from src.vectorizer.w2v_vectorizer import W2vVectorizer
from src.models.abstract_model import AbstractModel
from src.vectorizer.lsi_vectorizer import LsiVectorizer
from src.vectorizer.sk_count_vectorizer import SkCountVectorizer
from src.vectorizer.tfidf_vectorizer import TfidfVectorizer
from src.post_tokenizer.phraser_merger import PhraserMerger
from src.post_tokenizer.null_post_tokenizer import NullPostTokenizer
from src.tokenize.sentence_piece_tokenizer import SentencePieceTokenizer
from src.tokenize.null_tokenizer import NullTokenizer
from src.tokenize.spacy_tokenizer import SpacyTokenizer
from src.preprocessing.simple_preprocessor import SimplePreprocessor

stage_config = {
    'preprocessing': {
        SimplePreprocessor: {
            'remove_citations': (True, False),
            'remove_duplicates': (True, False),
        }
    },
    'tokenizer': {
        SpacyTokenizer: {
            'replace_numbers': (True, False),
            'remove_stopwords': (True, False),
        },
        NullTokenizer: {},
        SentencePieceTokenizer: {
            'vocab_size': (5000, 10000),
        },
    },
    'post_tokenizer': {
        NullPostTokenizer: {},
        PhraserMerger: {
            'num_gram': (1, 2),
        },
    },
    'vectorizer': {
        TfidfVectorizer: {},
        SkCountVectorizer: {
            'ignore_preprocessing': (True, False),
        },
        W2vVectorizer: {},
        LsiVectorizer: {},
        # FastTextW2vVectorizer: {},
    },
    'model': {
        AbstractModel: {},
    }
}

In [3]:
from itertools import product
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression


def run_preprocessing(run_class, args, prev_output, current_state, current_models):
    print(f'running preprocessing: {run_class}: {args=}, {current_state=}')
    current_obj = run_class(**args)
    preprocessed_train = current_obj.preprocess(train_data)
    preprocessed_dev = current_obj.preprocess(dev_data)
    preprocessed_test = current_obj.preprocess(test_data)
    return (preprocessed_train, preprocessed_dev, preprocessed_test), current_obj


def run_tokenizer(run_class, args, prev_output, current_state, current_models):
    print(f'running tokenizer: {run_class}: {args=}, {current_state=}')
    tokenizer = run_class(
        **args
    )
    train, dev, test = prev_output
    tokenizer.fit(train)
    results = tuple(tokenizer.tokenize(x) for x in prev_output)
    return results, tokenizer


def run_post_tokenizer(run_class, args, prev_output, current_state, current_models):
    print(f'running post_tokenizer: {run_class}: {args=}, {current_state=}')
    post_tokenizer = run_class(**args)
    train, dev, test = prev_output
    post_tokenizer.fit(train)
    results = tuple(post_tokenizer.transform(x) for x in prev_output)
    return results, post_tokenizer


def run_vectorizer(run_class, args, prev_output, current_state, current_models):
    print(f'running vectorizer: {run_class}: {args=}, {current_state=}')
    vectorizer = run_class(**args)
    train, dev, test = prev_output
    vectorizer.fit(train)
    results = tuple(vectorizer.transform(x) for x in prev_output)
    return results, vectorizer


def run_model(run_class, args, prev_output, current_state, current_models):
    print(f'running model: {run_class}: {args=}, {current_state=}')
    models = {
        'LR': LogisticRegression(max_iter=2000),
        # 'svm': SVC(kernel='rbf'),
    }
    train, dev, test = prev_output

    results = []
    for model_name, model in models.items():
        model_result = {'model_name': model_name}
        model.fit(train.vectors, train.label_indices)
        y_pred_train = model.predict(train.vectors)
        score = f1_score(train.label_indices, y_pred_train, average='macro')
        model_result['train_f1'] = score

        y_pred_dev = model.predict(dev.vectors)
        score = f1_score(dev.label_indices, y_pred_dev, average='macro')
        model_result['dev_f1'] = score

        y_pred_test = model.predict(test.vectors)
        score = f1_score(test.label_indices, y_pred_test, average='macro')
        model_result['test_f1'] = score
        results.append(model_result)
        print(f'{model_result=}')

    return results, None


def run(stage: str, run_class, args: dict, prev_output=None, current_state=None, current_models=None) -> list[dict]:
    if current_state is None:
        current_state = dict()
    current_state = {**current_state, stage: run_class.__name__}
    if current_models is None:
        current_models = dict()
    results = []
    next_stage = stage_order_map.get(stage)

    run_func = {
        'pretokenizer': run_preprocessing,
        'tokenizer': run_tokenizer,
        'post_tokenizer': run_post_tokenizer,
        'vectorizer': run_vectorizer,
        'model': run_model,
    }

    if not args:
        result, model = run_func[stage](run_class, args, prev_output, current_state, current_models)
        new_models = {**current_models, stage: model}
        if next_stage is not None:
            for next_class, next_args in stage_config[next_stage].items():
                new_state = current_state
                run_result = run(next_stage, next_class, next_args, result, new_state, new_models)
                results.extend(run_result)
            return results
        else:
            result_state = [{**current_state, **result_row} for result_row in result]
            return result_state

    argument_permutations = list(product(*args.values()))
    for values in argument_permutations:
        new_arg = dict(zip(args.keys(), values))
        result, model = run_func[stage](run_class, new_arg, prev_output, current_state, current_models)
        new_models = {**current_models, stage: model}
        if next_stage is not None:
            for next_class, next_args in stage_config[next_stage].items():
                new_state = {**current_state, **new_arg}
                run_result = run(next_stage, next_class, next_args, result, new_state, new_models)
                results.extend(run_result)
        else:
            result_state = [{**current_state, **result_row} for result_row in result]
            results.extend(result_state)
    return results



In [4]:
results = run('pretokenizer', SimplePreprocessor, {
    'remove_citations': (True, False),
    'remove_duplicates': (True, False),
}, None)
results_df = pd.DataFrame(results)
results_df

running preprocessing: <class 'src.preprocessing.simple_preprocessor.SimplePreprocessor'>: args={'remove_citations': True, 'remove_duplicates': True}, current_state={'pretokenizer': 'SimplePreprocessor'}
running tokenizer: <class 'src.tokenize.spacy_tokenizer.SpacyTokenizer'>: args={'replace_numbers': True, 'remove_stopwords': True}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': True, 'remove_duplicates': True, 'tokenizer': 'SpacyTokenizer'}
running post_tokenizer: <class 'src.post_tokenizer.null_post_tokenizer.NullPostTokenizer'>: args={}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': True, 'remove_duplicates': True, 'tokenizer': 'SpacyTokenizer', 'replace_numbers': True, 'remove_stopwords': True, 'post_tokenizer': 'NullPostTokenizer'}
running vectorizer: <class 'src.vectorizer.tfidf_vectorizer.TfidfVectorizer'>: args={}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': True, 'remove_duplicates': True, 'toke

C:\Users\lee_m\.conda\envs\cs4248_project\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


running model: <class 'src.models.abstract_model.AbstractModel'>: args={}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': True, 'remove_duplicates': True, 'tokenizer': 'SpacyTokenizer', 'replace_numbers': True, 'remove_stopwords': True, 'post_tokenizer': 'NullPostTokenizer', 'vectorizer': 'SkCountVectorizer', 'ignore_preprocessing': True, 'model': 'AbstractModel'}


C:\Users\lee_m\.conda\envs\cs4248_project\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


model_result={'model_name': 'LR', 'train_f1': 0.8067378232163117, 'dev_f1': 0.6740916345794465, 'test_f1': 0.6288266039004315}
running vectorizer: <class 'src.vectorizer.sk_count_vectorizer.SkCountVectorizer'>: args={'ignore_preprocessing': False}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': True, 'remove_duplicates': True, 'tokenizer': 'SpacyTokenizer', 'replace_numbers': True, 'remove_stopwords': True, 'post_tokenizer': 'NullPostTokenizer', 'vectorizer': 'SkCountVectorizer'}
running model: <class 'src.models.abstract_model.AbstractModel'>: args={}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': True, 'remove_duplicates': True, 'tokenizer': 'SpacyTokenizer', 'replace_numbers': True, 'remove_stopwords': True, 'post_tokenizer': 'NullPostTokenizer', 'vectorizer': 'SkCountVectorizer', 'ignore_preprocessing': False, 'model': 'AbstractModel'}
model_result={'model_name': 'LR', 'train_f1': 0.9998007816475948, 'dev_f1': 0.7531704588618823,

...

C:\Users\lee_m\.conda\envs\cs4248_project\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


running model: <class 'src.models.abstract_model.AbstractModel'>: args={}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': False, 'remove_duplicates': False, 'tokenizer': 'SentencePieceTokenizer', 'vocab_size': 10000, 'post_tokenizer': 'PhraserMerger', 'num_gram': 2, 'vectorizer': 'SkCountVectorizer', 'ignore_preprocessing': True, 'model': 'AbstractModel'}


C:\Users\lee_m\.conda\envs\cs4248_project\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


model_result={'model_name': 'LR', 'train_f1': 0.9029985217310532, 'dev_f1': 0.7179738907539507, 'test_f1': 0.7179934642884639}
running vectorizer: <class 'src.vectorizer.sk_count_vectorizer.SkCountVectorizer'>: args={'ignore_preprocessing': False}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': False, 'remove_duplicates': False, 'tokenizer': 'SentencePieceTokenizer', 'vocab_size': 10000, 'post_tokenizer': 'PhraserMerger', 'num_gram': 2, 'vectorizer': 'SkCountVectorizer'}
running model: <class 'src.models.abstract_model.AbstractModel'>: args={}, current_state={'pretokenizer': 'SimplePreprocessor', 'remove_citations': False, 'remove_duplicates': False, 'tokenizer': 'SentencePieceTokenizer', 'vocab_size': 10000, 'post_tokenizer': 'PhraserMerger', 'num_gram': 2, 'vectorizer': 'SkCountVectorizer', 'ignore_preprocessing': False, 'model': 'AbstractModel'}
model_result={'model_name': 'LR', 'train_f1': 0.9996787711424727, 'dev_f1': 0.7885875102349228, 'test_f1': 0.7930

,pretokenizer,remove_citations,remove_duplicates,tokenizer,replace_numbers,remove_stopwords,post_tokenizer,vectorizer,model,model_name,train_f1,dev_f1,test_f1,ignore_preprocessing,num_gram,vocab_size
0,SimplePreprocessor,True,True,SpacyTokenizer,True,True,NullPostTokenizer,TfidfVectorizer,AbstractModel,LR,0.870372,0.736147,0.723453,NaN,NaN,NaN
1,SimplePreprocessor,True,True,SpacyTokenizer,True,True,NullPostTokenizer,SkCountVectorizer,AbstractModel,LR,0.806738,0.674092,0.628827,True,NaN,NaN
2,SimplePreprocessor,True,True,SpacyTokenizer,True,True,NullPostTokenizer,SkCountVectorizer,AbstractModel,LR,0.999801,0.753170,0.737456,False,NaN,NaN
3,SimplePreprocessor,True,True,SpacyTokenizer,True,True,NullPostTokenizer,W2vVectorizer,AbstractModel,LR,0.663951,0.631870,0.635406,NaN,NaN,NaN
4,SimplePreprocessor,True,True,SpacyTokenizer,True,True,NullPostTokenizer,LsiVectorizer,AbstractModel,LR,0.728142,0.710348,0.690148,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,SimplePreprocessor,False,False,SentencePieceTokenizer,NaN,NaN,PhraserMerger,TfidfVectorizer,AbstractModel,LR,0.901645,0.781128,0.792285,NaN,2.0,10000.0
416,SimplePreprocessor,False,False,SentencePieceTokenizer,NaN,NaN,PhraserMerger,SkCountVectorizer,AbstractModel,LR,0.902999,0.717974,0.717993,True,2.0,10000.0
417,SimplePreprocessor,False,False,SentencePieceTokenizer,NaN,NaN,PhraserMerger,SkCountVectorizer,AbstractModel,LR,0.999679,0.788588,0.793014,False,2.0,10000.0
418,SimplePreprocessor,False,False,SentencePieceTokenizer,NaN,NaN,PhraserMerger,W2vVectorizer,AbstractModel,LR,0.370393,0.366388,0.337872,NaN,2.0,10000.0


In [5]:
from src.utils.path_getter import PathGetter

results_df.to_parquet(PathGetter.get_data_directory()/'experiments.parquet')

In [6]:
results_df.sort_values(by=['test_f1'], ascending=False)

,pretokenizer,remove_citations,remove_duplicates,tokenizer,replace_numbers,remove_stopwords,post_tokenizer,vectorizer,model,model_name,train_f1,dev_f1,test_f1,ignore_preprocessing,num_gram,vocab_size
170,SimplePreprocessor,True,False,NullTokenizer,NaN,NaN,PhraserMerger,TfidfVectorizer,AbstractModel,LR,0.901645,0.785547,0.811778,NaN,1.0,NaN
65,SimplePreprocessor,True,True,NullTokenizer,NaN,NaN,PhraserMerger,TfidfVectorizer,AbstractModel,LR,0.900072,0.782403,0.809262,NaN,1.0,NaN
380,SimplePreprocessor,False,False,NullTokenizer,NaN,NaN,PhraserMerger,TfidfVectorizer,AbstractModel,LR,0.904953,0.788257,0.808687,NaN,1.0,NaN
62,SimplePreprocessor,True,True,NullTokenizer,NaN,NaN,NullPostTokenizer,SkCountVectorizer,AbstractModel,LR,0.999801,0.800148,0.808455,False,NaN,NaN
70,SimplePreprocessor,True,True,NullTokenizer,NaN,NaN,PhraserMerger,TfidfVectorizer,AbstractModel,LR,0.896358,0.779570,0.806851,NaN,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,SimplePreprocessor,False,False,SentencePieceTokenizer,NaN,NaN,PhraserMerger,W2vVectorizer,AbstractModel,LR,0.370393,0.366388,0.337872,NaN,2.0,10000.0
293,SimplePreprocessor,False,True,SentencePieceTokenizer,NaN,NaN,PhraserMerger,W2vVectorizer,AbstractModel,LR,0.358704,0.355332,0.334485,NaN,1.0,5000.0
193,SimplePreprocessor,True,False,SentencePieceTokenizer,NaN,NaN,PhraserMerger,W2vVectorizer,AbstractModel,LR,0.347664,0.320911,0.319209,NaN,2.0,5000.0
403,SimplePreprocessor,False,False,SentencePieceTokenizer,NaN,NaN,PhraserMerger,W2vVectorizer,AbstractModel,LR,0.299330,0.306093,0.274905,NaN,2.0,5000.0
